## ebs.ErrorBudget.run_etc Required Arguments
### Fixed:  
* output_dir  
* ref_json_filename  
* pp_json_filename  
* contrast_filename  
* target_list  
* luminosity  
* eeid  
* eepsr  
* npoints 

### Non-variable, mutable  
### Possibly variable  
* exo_zodi  
* wfe
* wfsc_factor
* sensitivity
* output_filename_prefix
* var_par
* *args  

In [1]:
import numpy as np
import utilities.log_pdf as pdf

In [2]:
pdf.uniform

<function utilities.log_pdf.uniform(x, lower_bound, upper_bound)>